# Decision Tree Implementation

In [1]:
from sklearn import datasets

In [2]:
df = datasets.load_iris()
X = df.data
Y = df.target

In [3]:
features = [0,1,2,3]

In [4]:
def makeLabelled(column):
    """ Marks a label to each row of a column """
    second_limit = column.mean()
    first_limit = 0.5*second_limit
    third_limit = 1.5*second_limit
    for i in range(0,len(column)):
        if(column[i] < first_limit):
            column[i] = 0
        elif (column[i] < second_limit):
            column[i] = 1
        elif (column[i] < third_limit):
            column[i] = 2
        else:
            column[i] = 3
    return column

In [5]:
for i in range(0,X.shape[-1]):
    X[:,i] = makeLabelled(X[:,i])

In [6]:
from math import log
import numpy as np

In [7]:
def find_unique_values(x, col):
    """ Returns unique values of a column of X data set"""
    return set([row[col] for row in x])

In [8]:
def find_entropy(y):
    """ Returns entropy of the current classes """
    dictionary = {}  # No of data points of each class is stored in a dictionary
    entropy = 0
    s = set(y)
    for i in s:
        dictionary[i] = (y == i).sum()
    for key in dictionary:
        prob = dictionary[key]/len(y) # probability of each class
        if(prob != 0):
            entropy += (-1)*prob*log(prob,2)
    return entropy

In [9]:
def find_info_gain(x,y,f):
    """ Returns Information gain on the basis of the given feature """
    parent_entropy = find_entropy(y)  # Entropy of parent node
    s = find_unique_values(x,f)
    result = {}      # result will store the entropy of each child
    dictionary = {}  # dictionary stores no of data points of each unique value of x[:,f]
    for e in s:
        rows = (x[:,f] == e)
        dictionary[e] = rows.sum()   # it will be used to find the weighted avg of children entropy
        result[e] = find_entropy(y[rows])
    weighted_child_entropy = 0      # this will store the weighted children entropy
    for key in result:
        weighted_child_entropy += (dictionary[key]/len(x))*(result[key])
    return parent_entropy - weighted_child_entropy

In [10]:
def base_case_pure_node(y,level):
    """ Base case for a pure node """
    print("Level:", level)
    print("Count of", y[0], "=", len(y))
    print("Current Entropy is = 0.0")
    print("Leaf Node Reached")
    print()

In [11]:
def base_case_no_features_left(y,level):
    """ Base case when no feature are left to split """
    print("Level:", level)
    s = set(y)
    for e in s:
        print("Count of", e, "=", (y == e).sum())
    print("Current Entropy is", find_entropy(y))
    print()

In [12]:
def print_current_node(y,max_gain,final_feature,level):
    """ Prints status of current node """
    print("Level:", level)
    s = set(y)
    for e in s:
        print("Count of", e, "=", (y == e).sum())
    print("Current Entropy is", find_entropy(y))
    print("Splitting on feature", final_feature, " with information gain", max_gain)
    print()

In [13]:
def find_best_split(x,y,features):
    """ Finds best split for the dataset on the basis of information gain
        Returns the maximum info_gain and the feature on which to split
    """
    max_gain = 0
    final_feature = -2 # assigned a random value which the final_feature cannot take
    for f in features:
        info_gain = find_info_gain(x,y,f)
        if(info_gain > max_gain):     # compares info_gain of each feature of x
            max_gain = info_gain
            final_feature = f
    return max_gain, final_feature

In [14]:
def partition(x,y,features,final_feature,level):
    """ Partition the dataset on the basis of feature split which has highest information gain """
    s = set(x[:,final_feature]) # Returns unique values of x for final_feature column
    features.remove(final_feature) # Deleted the feature on which the split has to be made
    for i in s:
        arr = (x[:,final_feature] == i)
        build_DT(x[arr], y[arr], features, level + 1)   # RECURSIVE CALL

In [15]:
def build_DT(x, y, features, level):
    """ This function builds the decision tree"""
    #base case for pure node
    if (len(set(y)) == 1):
        base_case_pure_node(y, level)
        return
    
    #base case when all features are used in splitting
    if(len(features) == 0):
        base_case_no_features_left(y, level)
        return
    
    #Decision to make on which feature to split on
    max_gain, final_feature = find_best_split(x, y, features)
    
    # Printing current node
    print_current_node(y, max_gain, final_feature, level)
    
    #Splitting the data on the basis of maximum gain
    partition(x, y, features, final_feature, level)

In [16]:
build_DT(X,Y,features,0)

Level: 0
Count of 0 = 50
Count of 1 = 50
Count of 2 = 50
Current Entropy is 1.58496250072
Splitting on feature 3  with information gain 1.35656522266

Level: 1
Count of 0 = 49
Current Entropy is = 0.0
Leaf Node Reached

Level: 1
Count of 0 = 1
Count of 1 = 10
Current Entropy is 0.439496986922
Splitting on feature 1  with information gain 0.439496986922

Level: 2
Count of 1 = 10
Current Entropy is = 0.0
Leaf Node Reached

Level: 2
Count of 0 = 1
Current Entropy is = 0.0
Leaf Node Reached

Level: 1
Count of 1 = 39
Count of 2 = 5
Current Entropy is 0.510787822954
Splitting on feature 2  with information gain 0.0776949537301

Level: 2
Count of 1 = 1
Current Entropy is = 0.0
Leaf Node Reached

Level: 2
Count of 1 = 38
Count of 2 = 4
Current Entropy is 0.453716339187
Splitting on feature 0  with information gain 0.00399336146638

Level: 3
Count of 1 = 14
Count of 2 = 1
Current Entropy is 0.353359335021

Level: 3
Count of 1 = 24
Count of 2 = 3
Current Entropy is 0.503258334776

Level: 2
Count